# Make input files for geoclaw_fgmax_test

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os,sys
from clawpack.visclaw import colormaps

In [ ]:
sys.path.insert(0,'../../new_python')
import region_tools, topotools, marching_front, dtopotools
import fgmax_tools
from plottools import pcolorcells

In [ ]:
zmin = -60.
zmax = 40.

land_cmap = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})

sea_cmap = colormaps.make_colormap({ 0.0:[0,0,1], 1.:[.8,.8,1]})

cmap, norm = colormaps.add_colormaps((land_cmap, sea_cmap),
                                     data_limits=(zmin,zmax),
                                     data_break=0.)
                                     
sea_cmap_dry = colormaps.make_colormap({ 0.0:[1.0,0.7,0.7], 1.:[1.0,0.7,0.7]})
cmap_dry, norm_dry = colormaps.add_colormaps((land_cmap, sea_cmap_dry),
                                     data_limits=(zmin,zmax),
                                     data_break=0.)

In [ ]:
import maketopo
maketopo.maketopo()
maketopo.makeqinit()

In [ ]:
island_topo = topotools.Topography(path='island.tt3', topo_type=3)

In [ ]:
island_topo.plot()

In [ ]:
extent = [14.25, 14.65, 50.1, 50.35]
topo = island_topo.crop(extent)

figure(figsize=(12,6))
pcolorcells(topo.X, topo.Y, topo.Z, cmap=cmap, norm=norm)
colorbar(extend='both', ticks=arange(zmin,zmax+1,10))
gca().set_aspect(1./cos(50.2*pi/180.))

In [ ]:
pts_chosen = marching_front.select_by_flooding(topo.Z, Z1=0, Z2=1e6, max_iters=10) 

In [ ]:
pts_chosen = marching_front.select_by_flooding(topo.Z, Z1=0, Z2=15., 
                                               prev_pts_chosen=pts_chosen,
                                               max_iters=None) 

In [ ]:
pts_chosen_shallow = marching_front.select_by_flooding(topo.Z, Z1=0, Z2=-15., max_iters=None) 
pts_chosen = logical_and(pts_chosen, pts_chosen_shallow)

In [ ]:
Zchosen = ma.masked_array(topo.Z, logical_not(pts_chosen))

In [ ]:
figure(figsize=(12,6))
pcolorcells(topo.X, topo.Y, Zchosen, cmap=cmap, norm=norm)
colorbar(extend='both', ticks=arange(zmin,zmax+1,10))
gca().set_aspect(1./cos(48*pi/180.))

In [ ]:
fname_fgmax_mask = 'fgmax_pts_island.data'
topo_fgmax_mask = topotools.Topography()
chosen = where(pts_chosen, 1, 0)
topo_fgmax_mask.set_xyZ(topo.x, topo.y, chosen)
topo_fgmax_mask.write(fname_fgmax_mask, topo_type=3, Z_format='%1i')
print('Created %s' % fname_fgmax_mask)